<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/Copy_of_Challenge_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-03 19:57:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  3.97MB/s    in 0.2s    

2021-09-03 19:57:27 (3.97 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2013+(UF).csv"), sep=",", header=True)


In [5]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY', 'EMPSTATE', 'PREVAMNT', 'REFUNDDATE', 'PAY_METHOD','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+----+---------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|               NAME|C_CODE|BOROUGHCD|             CITY|STATE|  ZIP|AMNT|MATCHAMNT|
+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+----+---------+
|    2013|       5|   1008|       P|Peterson, Daniel P|3/11/2012|   Haywood, David B|   IND|        X|            Bronx|   NY|10458|  20|       20|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|      Herlihy, Neil|   IND|        Z|       Hunnington|   NY|11746|  25|        0|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/5/2012|       Iocca, Karen|   IND|        Z|           Hobken|   NJ| 7030|  25|        0|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|   Lewenberg, Ilana|   IND|        M|         N

In [6]:
df.select('C_CODE').distinct().show(truncate=False)

+------+
|C_CODE|
+------+
|EMPO  |
|PCOMZ |
|null  |
|FAM   |
|ORG   |
|LLC   |
|PCOMP |
|CAN   |
|SPO   |
|OTHR  |
|PCOMC |
|IND   |
|CORP  |
|PART  |
+------+



In [12]:
from pyspark.sql.functions import regexp_replace
df1 = df.withColumn('C_CODE', regexp_replace('C_CODE', 'CAN', 'Candidate')) \
  .withColumn('C_CODE', regexp_replace('C_CODE', 'CORP', 'Corporation')) \
  .withColumn('C_CODE', regexp_replace('C_CODE', 'EMPO', 'Labor Union')) \
  .withColumn('C_CODE', regexp_replace('C_CODE', 'FAM', 'Candidate Family')) \
  .withColumn('C_CODE', regexp_replace('C_CODE', 'IND', 'Individual')) \

#from pyspark.sql.functions import regexp_replace
#df.withColumn('C_CODE', regexp_replace('Contributor_Type', 'IND', 'Individual')) \
 # .show(truncate=False)

In [13]:
#Check the CANCLASS is only P
df1.select('C_CODE').distinct().show(truncate=False)

+----------------+
|C_CODE          |
+----------------+
|PCOMZ           |
|Labor Union     |
|null            |
|Individual      |
|ORG             |
|Candidate Family|
|LLC             |
|Candidate       |
|PCOMP           |
|SPO             |
|Corporation     |
|OTHR            |
|PCOMC           |
|PART            |
+----------------+



In [14]:
df3 = df1.filter((df1.OFFICECD=="1") & (df1.CANCLASS=="P"))

In [15]:
df3.show()

+--------+--------+-------+--------+--------------------+----------+-------------------+----------+---------+--------------+-----+-----+----+---------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|           RECIPNAME|      DATE|               NAME|    C_CODE|BOROUGHCD|          CITY|STATE|  ZIP|AMNT|MATCHAMNT|
+--------+--------+-------+--------+--------------------+----------+-------------------+----------+---------+--------------+-----+-----+----+---------+
|    2013|       1|   1286|       P|Coenen, Jr., Kevin P| 1/25/2013|    Coenen, Kevin P| Candidate|        Z|     Nesconset|   NY|11767|  10|        0|
|    2013|       1|   1286|       P|Coenen, Jr., Kevin P| 4/17/2013|     Walter, Dorsey|Individual|        M|      New York|   NY|10150| 175|      175|
|    2013|       1|   1286|       P|Coenen, Jr., Kevin P| 1/25/2013|    Coenen, Kevin P| Candidate|        Z|     Nesconset|   NY|11767| -10|        0|
|    2013|       1|   1551|       P|     Marks, Philip A| 2/13/2012|      Marks, Philip|

In [16]:
df3.select('C_CODE').distinct().show(truncate=False)

+----------------+
|C_CODE          |
+----------------+
|PCOMZ           |
|Labor Union     |
|Individual      |
|Candidate Family|
|LLC             |
|Candidate       |
|PCOMP           |
|SPO             |
|Corporation     |
|OTHR            |
|PCOMC           |
|PART            |
+----------------+

